# The Battle of Neighborhoods
### Opening a restaurant in Lyon
## Description of the problem

Lyon is one of the biggest French cities. Located inside the country it has two rivers flowing inside: the Saône and the Rhône which joins to become the Rhône. In addition, this city has many hills which gives it a unique look.



Inside the city center are many restaurant, cafés and bars, especially on the “presqu’île” (area between the two rivers before they merge). At night the city is crowded by people looking to have a good time. There are many different types of restaurant, the most typical being the “bouchon Lyonnais”.


I’ve got a friend who would like to open a restaurant in the city. It would be a restaurant of American food since he is born in the US. He also believes that American food restaurant are not widespread in this area which would give his business a competitive advantage.
However, he is unsure of where to install his business inside the city. 


The goal of this project is then to give him a set of different places where he could install his new restaurant as well as a set of different indicators helping him to understand the advantages of each area in order to adapt his offer. (attendance, number of restaurants installed, tourism activity, …).

## Description of the data and how to use it

I will use the Foursquare data in this project. It contains a certain number of restaurants located in Lyon. For each restaurant we have access to different type of information, namely the type of food, the price, their score, reviews etc…



I will retrieve the data using the methods learned in this course. I will then try to identify different areas using clustering techniques. Having these areas, I will then try to determine specific characteristics for each using different indicators and charts.

Once those areas and characteristics are defined, I will make a selection to determine which areas are the best to install a new restaurant.
This will be the result I will hand to my friend so he can investigate those areas by himself.

### Examples of data

Let's have a look at the type af data I will use. For now it is only data from Foursquare.

In [75]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

In [52]:
CLIENT_ID = 'SJ5ILVLRNBXCWC30C1WKSNCLH2JQVDW451GEMOJ4S4Z3NWLI' # your Foursquare ID
CLIENT_SECRET = 'YFGB1YKWXORVA3TUOU3O1RN5TRMZDVHD5L54BMXHQMKYHE1E' # your Foursquare Secret
VERSION = '20200301'
CITY = 'Lyon'
QUERY = 'restaurant'
LIMIT = '90'

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&query={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION,CITY,QUERY,LIMIT)

In [53]:
results = requests.get(url).json()

In [54]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']     
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [57]:
venues = results['response']['groups'][0]['items']

nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row

nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Le Musée,Lyonese Bouchon,45.764704,4.834464
1,Le Petit Carron,Lyonese Bouchon,45.752880,4.852723
2,Puzzle café,Café,45.764206,4.833891
3,Le Tire Bouchon,Lyonese Bouchon,45.762715,4.827210
4,À Chacun sa Tasse,Café,45.769489,4.835852


We now have access to 90 locations of restaurants in Lyon with their categories. It is enough to give us a first good look. But we can do more

The city of Lyon has made accessible a data base for tourism activities on its website: https://data.grandlyon.com/

I will use one of the databases available to get more information on the restaurants and possibly to merge this information with the foursquare data.

In [80]:
url = 'https://download.data.grandlyon.com/ws/rdata/sit_sitra.sittourisme/all.json?maxfeatures=5236&start=1'
results = requests.get(url).json()

In [86]:
resto_lyon = json_normalize(results['values'])
resto_lyon = resto_lyon[['type','nom','adresse','type_detail']]
resto_lyon['type'].unique()
resto_lyon = resto_lyon[resto_lyon['type'] == 'RESTAURATION']
resto_lyon = resto_lyon[resto_lyon['adresse'] != '']
resto_lyon = resto_lyon[resto_lyon['type_detail'] != '']
resto_lyon['type_detail'].unique()

array(['Restaurant traditionnel', 'Restaurant gastronomique',
       'Restaurant de spécialités étrangères',
       'Restaurant traditionnel;Restaurant bistronomique',
       'Restaurant bistronomique', 'Brasserie;Restaurant traditionnel',
       'Restauration rapide',
       'Restaurant traditionnel;Restaurant gastronomique', 'Brasserie',
       'Brasserie;Restaurant traditionnel;Restaurant spectacle;Restaurant dansant',
       'Restaurant traditionnel;Grill / Rôtisserie',
       'Restaurant traditionnel;Bouchon',
       'Restaurant gastronomique;Restaurant bistronomique', 'Bouchon',
       'Grill', 'Pizzeria',
       'Brasserie;Restaurant de spécialités étrangères',
       'Brasserie;Pizzeria;Restauration rapide',
       'Restauration rapide;Restaurant de spécialités étrangères',
       'Grill / Rôtisserie', "Restaurant d'application",
       'Restaurant traditionnel;Grill',
       'Grill / Rôtisserie;Restaurant de spécialités étrangères',
       'Restaurant spectacle', 'Restaurant t

In [89]:
resto_lyon.head()

,type,nom,adresse,type_detail
4,RESTAURATION,Les Petites Cantines Vaise,37 rue Saint-Pierre de Vaise,Restaurant traditionnel
13,RESTAURATION,Le Jardin du Pré fleuri,240 rue Francis de Pressensé,Restaurant traditionnel
22,RESTAURATION,Nosch Cantine et Comptoir,24 rue Palais Grillet,Restaurant traditionnel
24,RESTAURATION,La Mutinerie,123 rue bugeaud,Restaurant gastronomique
25,RESTAURATION,Le Crusoé,Quai Illhaeusern,Restaurant traditionnel


In [90]:
resto_lyon.shape

(1087, 4)

This database gives us way more restaurants than Foursquare does but the type of information regarding the type of restaurant is different. This database does not give the cuisine the same way than Foursquare does. We will not be able to cluster them with the Foursquare data. But we will be able to cluster them independetly to get other type of information on the areas. Also this data need to be put into proper form, as of right now it cannot be used.